In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install transformers[torch] tokenizers datasets evaluate rouge_score sentencepiece huggingface_hub --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-no

In [3]:
import nltk
import evaluate
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [4]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
no_finetuned = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
inputs = tokenizer("What can we make with beef and tomato and what is the recipe", return_tensors="pt")
outputs = no_finetuned.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['Tomato soup']


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [6]:
finetuned = T5ForConditionalGeneration.from_pretrained("gdrive/My Drive/T5_5000_v2", local_files_only=True)

In [7]:
outputs = finetuned.generate(**inputs, max_length=512)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

["you can make Beef And Tomato Casserole with 1 lb. beef and tomato and 1 can tomato and tomato soup mix. Here's the instruction : Brown beef and tomato in skillet. Add beef and tomato soup mix. Cook until beef is tender. Add soup mix and tomato soup mix. Cook until thickened. Serve with rice."]


In [14]:
inputs = tokenizer("answer the question: How to make Kung-Pao Chicken?", return_tensors="pt")
outputs = no_finetuned.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['Preheat the oven to 375 degrees F. Place the chicken in a large bowl.']


In [15]:
outputs = finetuned.generate(**inputs, max_length=512)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

["you can make Kung-Pao Chicken with 1 chicken, cooked and drained, 1 can Kung-Pao Sauce, 1 can Kung-Pao Chicken, 1 can Kung-Pao Sauce, 1 can Kung-Pao Chicken, 1 can Kung-Pao Chicken. Here's the instruction : Cook chicken in a large skillet until done. Drain and set aside. In a large bowl, combine all ingredients. Pour into a greased and floured 9 x 13-inch pan. Bake at 350° for 30 minutes."]


In [16]:
inputs = tokenizer("answer the question: What can we make with potato, salmon, and Broccoli and what is the recipe", return_tensors="pt")
outputs = no_finetuned.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['salmon']


In [17]:
outputs = finetuned.generate(**inputs, max_length=512)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

["you can make Salmon Casserole with 1 lb. salmon, 1 lb. Broccoli, 1 lb. potatoes, 1 lb. Broccoli and potatoes, 1 c. cooked salmon, 1 c. cooked Broccoli, 1 c. cooked potatoes. Here's the instruction : Cook salmon in boiling water until tender. Drain and set aside. In a large bowl, combine all ingredients. Pour into a 9 x 13-inch baking dish. Bake at 350° for 30 minutes or until done."]
